In [1]:
#Import Libraries
import requests
import os
import csv
import zipfile
import openpyxl
import sqlite3
import glob
import getpass
import codecs
import pandas as pd
import urllib.request
import numpy as np

In [2]:
#URL to get the dataset
url="https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"

In [3]:
r = requests.get(url)

In [4]:
#Creating a staging subdirectory called “staging”
staging_dir_name = "staging"
if not os.path.isdir(staging_dir_name):
    os.mkdir(staging_dir_name)

In [5]:
#Check if staging subdirectory is created
if os.path.isdir(staging_dir_name):
    zip_file_name = os.path.join(staging_dir_name, "test.zip")
    zf = open(zip_file_name,"wb")
    zf.write(r.content)
    zf.close()

In [6]:
#Extract all files from the zipped file
z = zipfile.ZipFile(zip_file_name,'r')
z.extractall(staging_dir_name)
z.close()

In [7]:
#Convert encoding type cp1252 to utf-8
for filename in glob.glob(os.path.join(staging_dir_name, "*.csv")):
    in_fp = open(filename,"rt",encoding='cp1252')
    input_data = in_fp.read()
    in_fp.close()
    
    out_fp = open(filename,"wt",encoding='utf-8')
    for c in input_data:
        if c!= '\0':
            out_fp.write(c)
    out_fp.close()

In [8]:
#Create DB
conn = sqlite3.connect("medicare_hospital_compare.db")
conn.text_factory = str  # allows utf-8 data to be stored
c = conn.cursor()

In [9]:
#This method gives all the csv files in the directory to the fileToTable method  
def CallFileToTable(dirname, db):
    for filename in glob.glob(os.path.join(dirname, '*.csv')):
        fileToTable(filename, db)

In [10]:
#Method to create a table for a csv file with hearers as column name
def fileToTable(filename, db):
        with open(filename, encoding="utf8") as f:
            with db:
                data = csv.DictReader(f)
                cols23 = data.fieldnames
                cols = list()
                #Replace blank space, -, / with _ and % with pct, convert to lower case
                table= (((str.lower(os.path.splitext(os.path.basename(filename))[0]).replace(" ", "_")).replace("-", "_")).replace("%", "pct")).replace('/', '_')
                if  not table[0].isalpha():
                    table = "t_" + table
                #Replace blank space, -, / with _ and % with pct, convert to lower case
                for col in cols23:
                    col = str.lower((((col.replace(" ", "_")).replace("-", "_")).replace("%", "pct")).replace('/', '_'))
                    #Prepend c_ if the initial letter of the column name is not an alphabet
                    if  not col[0].isalpha():
                        col = "c_" + col
                    cols.append(col)
                
                #If a table already exists, drop it                  
                sql = 'drop table if exists "{}"'.format(table)
                db.execute(sql)
                
                #Create Table 
                sql = 'create table "{table}" ( {cols} )'.format(
                    table=table,
                    cols=','.join('"{}"'.format(col) for col in cols))
                db.execute(sql)
                
                #Insert data into tables
                sql = 'insert into "{table}" values ( {vals} )'.format(
                    table=table,
                    vals=','.join('?' for col in cols))
                db.executemany(sql, (list(map(row.get, cols23)) for row in data))

In [11]:
#Create table for each CSV file
CallFileToTable(staging_dir_name, conn)

In [12]:
conn.commit()

In [13]:
k_url = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"

In [14]:
r = requests.get(k_url)

In [15]:
xf = open("hospital_ranking_focus_states.xlsx", "wb")

In [16]:
xf.write(r.content)

85117

In [17]:
# Import the excel file and call it xls_file
xls_file = pd.ExcelFile('hospital_ranking_focus_states.xlsx')

In [18]:
# Load the xls file's Sheet1 as a dataframe
df = xls_file.parse(xls_file.sheet_names[0])

In [19]:
df = df.sort_values(list(df)[1], ascending=True)

In [20]:
dff = df.astype(str)

In [21]:
dfalldata = (pd.read_sql_query("select provider_id as [Provider ID], hospital_name as [Hospital Name], city as City, state as State, county_name as County from hospital_general_information;", conn)).astype(str)

In [22]:
ctr = 0
while ctr < 1000:
    a = int(dff['Provider ID'].iloc[[ctr]].str.len())
    if a == 5:
        dff['Provider ID'].iloc[[ctr]] = '0' + dff['Provider ID'].iloc[[ctr]]
    ctr += 1

In [23]:
df1 = dff.head(1000)

In [24]:
df2 = pd.merge(df1, dfalldata, on ='Provider ID', how='inner')

In [25]:
df3 = df2[['Provider ID', 'Hospital Name', 'City', 'State', 'County']]

In [26]:
#######################################
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('hospital_ranking.xlsx', engine='xlsxwriter')

In [27]:
# Convert the dataframe to an XlsxWriter Excel object.
df3.to_excel(writer, sheet_name='Nationwide', index = False)

In [28]:
df4 = xls_file.parse(xls_file.sheet_names[1])

In [29]:
df5 = pd.merge(dff, dfalldata, on ='Provider ID', how='left')

In [30]:
df6 = df5[['Provider ID', 'Hospital Name', 'City', 'State', 'County']]

In [33]:
for index, row in df4.iterrows():
    df7 = df6.loc[df6['State'] == row['State Abbreviation']]
    df7 = df7.head(100)
    # Convert the dataframe to an XlsxWriter Excel object.
    df7.to_excel(writer, sheet_name=row['State Name'], index = False)

In [34]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [35]:
pd.options.mode.chained_assignment = None  # default='warn'

In [36]:
k_url = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"

In [37]:
r = requests.get(k_url)

In [38]:
xf = open("hospital_ranking_focus_states.xlsx", "wb")

In [40]:
xf.write(r.content)

85117

In [41]:
# Import the excel file and call it xls_file
xls_file = pd.ExcelFile('hospital_ranking_focus_states.xlsx')

In [42]:
dfNationalMeasureData = (pd.read_sql_query("select state as State, measure_id as [Measure ID], measure_name as [Measure Name], score as Score from timely_and_effective_care___hospital order by measure_id;", conn))

In [43]:
dfNational = (pd.read_sql_query("select distinct measure_id as [Measure ID], measure_name as [Measure Name] from timely_and_effective_care___hospital order by [Measure ID];", conn))

In [44]:
# Convert Series datatype to numeric, getting rid of any non-numeric values
dfNation = dfNationalMeasureData[dfNationalMeasureData[['Score']].apply(pd.to_numeric, errors='coerce').notnull().all(axis=1)]

In [45]:
dfNationwide = dfNation[['Measure ID','Score']]
dfNationwide['Score'] = dfNationwide['Score'].astype(float)
dfmean = dfNationwide.groupby(['Measure ID']).mean()
dfmin = dfNationwide.groupby(['Measure ID']).min()
dfmax = dfNationwide.groupby(['Measure ID']).max()
dfstd = dfNationwide.groupby(['Measure ID']).std()

In [46]:
dfmean = dfmean.rename(columns = {'Score':'Average'})
dfmin = dfmin.rename(columns = {'Score':'Minimum'})
dfmax = dfmax.rename(columns = {'Score':'Maximum'})
dfstd = dfstd.rename(columns = {'Score':'Standard Deviation'})

In [47]:
dfMeasure = pd.merge(dfNational, dfmin.reset_index(), on ='Measure ID', how='inner')
dfMeasure = pd.merge(dfMeasure, dfmax.reset_index(), on ='Measure ID', how='inner')
dfMeasure = pd.merge(dfMeasure, dfmean.reset_index(), on ='Measure ID', how='inner')
dfMeasure = pd.merge(dfMeasure, dfstd.reset_index(), on ='Measure ID', how='inner')

In [48]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('measures_statistics.xlsx', engine='xlsxwriter')

In [49]:
# Convert the dataframe to an XlsxWriter Excel object.
dfMeasure.to_excel(writer, sheet_name='Nationwide', index = False)

In [50]:
newdf4 = xls_file.parse(xls_file.sheet_names[1])

In [51]:
for index, row in newdf4.iterrows():
    newdfNation = dfNation.loc[dfNation['State'] == row['State Abbreviation']]
    newdfNationwide = newdfNation[['Measure ID','Score']]
    newdfNationwide['Score'] = newdfNationwide['Score'].astype(float)
    newdfmean = newdfNationwide.groupby(['Measure ID']).mean()
    newdfmin = newdfNationwide.groupby(['Measure ID']).min()
    newdfmax = newdfNationwide.groupby(['Measure ID']).max()
    newdfstd = newdfNationwide.groupby(['Measure ID']).std()
    
    newdfmean = newdfmean.rename(columns = {'Score':'Average'})
    newdfmin = newdfmin.rename(columns = {'Score':'Minimum'})
    newdfmax = newdfmax.rename(columns = {'Score':'Maximum'})
    newdfstd = newdfstd.rename(columns = {'Score':'Standard Deviation'})
    
    newdfMeasure = pd.merge(dfNational, newdfmin.reset_index(), on ='Measure ID', how='inner')
    newdfMeasure = pd.merge(newdfMeasure, newdfmax.reset_index(), on ='Measure ID', how='inner')
    newdfMeasure = pd.merge(newdfMeasure, newdfmean.reset_index(), on ='Measure ID', how='inner')
    newdfMeasure = pd.merge(newdfMeasure, newdfstd.reset_index(), on ='Measure ID', how='inner')
    
    # Convert the dataframe to an XlsxWriter Excel object.
    newdfMeasure.to_excel(writer, sheet_name=row['State Name'], index = False)
    
#####################
# Close the Pandas Excel writer and output the Excel file.
writer.save()